In [11]:
from loguru import logger
import json
import pandas as pd
from pandera import Column, DataFrameSchema
from pandera.errors import SchemaError

from ppgmne_prf.config.paths import PATH_DATA_PRF
from ppgmne_prf.utils import csv_zip_to_df


verbose = True

In [12]:
def read_accidents(urls: dict):
    key_in = "id"

    df_schema: dict[str, Column] = {
    "id": Column(int),
    "ano": Column(int),
    "data_inversa": Column(str),
    "dia_semana": Column(str),
    "horario": Column(str),
    "uf": Column(str),
    "municipio": Column(str),
    "causa_acidente": Column(str),
    "tipo_acidente": Column(str, nullable=True),
    "classificacao_acidente": Column(str),
    "fase_dia": Column(str),
    "sentido_via": Column(str),
    "tipo_pista": Column(str),
    "tracado_via": Column(str),
    "uso_solo": Column(str),
    "pessoas": Column(int),
    "mortos": Column(int),
    "feridos_leves": Column(int),
    "feridos_graves": Column(int),
    "ilesos": Column(int),
    "ignorados": Column(int),
    "feridos": Column(int),
    "veiculos": Column(int),
    "latitude": Column(str),
    "longitude": Column(str),
    "regional": Column(str, nullable=True),
    "delegacia": Column(str, nullable=True),
    "uop": Column(str, nullable=True),
    }

    df_out = pd.DataFrame()
    for year in urls.keys():
        # Lê os dados dos acidentes:
        url = urls[year]
        file_name = f"datatran{year}.csv"
        df = csv_zip_to_df(url, file_name)
        df["ano"] = year

        logger.info(f"Lendo os registros de acidentes de {year}.") if verbose else None
        df_out = pd.DataFrame()  
        # Valida os dados de entrada:
        try:
            df = DataFrameSchema(
                columns=df_schema,
                unique=key_in,
                coerce=True,
                strict="filter",
            ).validate(df)
        except SchemaError as se:
            logger.error(f"Erro ao validar os dados dos acidentes de {year}.")
            logger.error(se)

        # Concatena os anos:
        if df_out.shape[0] == 0:
            df_out = df.copy()
        else:
            df_out = pd.concat([df_out, df], ignore_index=True)

    return df_out
    

In [13]:
logger.info("Lendo as urls dos acidentes.") if verbose else None
with open(PATH_DATA_PRF / "accidents.json") as file:
    urls = json.load(file)

df_accidents = read_accidents(urls)

2022-12-11 18:09:21.557 | INFO     | __main__:<module>:1 - Lendo as urls dos acidentes.
2022-12-11 18:09:23.718 | INFO     | __main__:read_accidents:43 - Lendo os registros de acidentes de 2018.
2022-12-11 18:09:25.886 | INFO     | __main__:read_accidents:43 - Lendo os registros de acidentes de 2019.
2022-12-11 18:09:27.988 | INFO     | __main__:read_accidents:43 - Lendo os registros de acidentes de 2020.
2022-12-11 18:09:30.209 | INFO     | __main__:read_accidents:43 - Lendo os registros de acidentes de 2021.
2022-12-11 18:09:32.079 | INFO     | __main__:read_accidents:43 - Lendo os registros de acidentes de 2022.


In [14]:
df_accidents.head()

,id,data_inversa,dia_semana,horario,uf,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,...,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,ano
0,405151,2022-01-01,sábado,01:35:00,PI,MARCOLANDIA,Ingestão de álcool pelo condutor,Colisão traseira,Com Vítimas Feridas,Plena Noite,...,1,0,1,2,"-7,43280012","-40,68261908",SPRF-PI,DEL04-PI,UOP03-DEL04-PI,2022
1,405158,2022-01-01,sábado,02:40:00,PR,CAMPINA GRANDE DO SUL,Ingestão de álcool pelo condutor,Tombamento,Com Vítimas Fatais,Pleno dia,...,0,0,1,1,"-25,114403","-48,8467554",SPRF-PR,DEL01-PR,UOP02-DEL01-PR,2022
2,405172,2022-01-01,sábado,05:22:00,MS,NOVA ALVORADA DO SUL,Condutor deixou de manter distância do veículo...,Colisão traseira,Com Vítimas Feridas,Amanhecer,...,1,0,1,2,"-21,228445","-54,456296",SPRF-MS,DEL02-MS,UOP01-DEL02-MS,2022
3,405203,2022-01-01,sábado,07:00:00,RJ,ANGRA DOS REIS,Reação tardia ou ineficiente do condutor,Colisão frontal,Com Vítimas Feridas,Pleno dia,...,1,0,1,2,"-23,03149804","-44,17715297",SPRF-RJ,DEL03-RJ,UOP02-DEL03-RJ,2022
4,405207,2022-01-01,sábado,09:00:00,MG,RIBEIRAO DAS NEVES,Acumulo de água sobre o pavimento,Saída de leito carroçável,Sem Vítimas,Pleno dia,...,3,0,0,1,"-19,76061249","-44,1347544",SPRF-MG,DEL02-MG,UOP01-DEL02-MG,2022
